In [ ]:
from pathlib import Path
import sys

import polars as pl


sys.path.append(str(Path.cwd().parent))

from libraries.client_stashapp import StashAppClient, get_stashapp_client


stash = get_stashapp_client()
stash_client = StashAppClient()

In [ ]:
stash_tags = stash_client.get_tags_by_names(["AI", "AI_Tagged"])

In [ ]:
scenes_to_be_reset = pl.DataFrame({ "id": "1216" })
scenes_to_be_reset

In [ ]:
for selected_scene in scenes_to_be_reset.iter_rows(named=True):
    scene_id = selected_scene["id"]
    fragment = (
        "id title tags { id name parents { id name } }"
        " scene_markers { id primary_tag { id name"
        " parents { id name } } tags { id name } }"
    )
    scene = stash.find_scene(scene_id, fragment=fragment)

    # Get all tags from the scene
    all_scene_tags = scene["tags"]
    ai_tags = [
        tag for tag in all_scene_tags
        if any(
            parent["id"] == stash_tags.ai["id"]
            for parent in tag["parents"]
        )
    ]
    ai_tag_ids = [tag["id"] for tag in ai_tags]
    stash_client.bulk_scene_update(
        [scene_id],
        ai_tag_ids + [stash_tags.ai_tagged["id"]],
        "REMOVE",
    )

    scene_markers = scene["scene_markers"]
    for marker in scene_markers:
        # IF any of the parents of the primary tag is AI, then remove the marker
        if any(
            parent["id"] == stash_tags.ai["id"]
            for parent in marker["primary_tag"]["parents"]
        ):
            stash.destroy_scene_marker(marker["id"])